# はじめてのシングルセル解析：JIA滑膜CITE-seqデータの解析 Part 2
### サブクラス解析（T細胞のさらなる分類）

**著者：** 稲毛 純（Inamo Jun）　慶應義塾大学医学部 微生物学・免疫学教室

---

## ▶ 毎回の開始手順（必ずお読みください）

### ステップ①：ランタイムを R に設定（初回のみ）
メニュー「ランタイム」→「ランタイムのタイプを変更」→ **R** を選択 → 保存

### ステップ②：2つのファイルをアップロード
左サイドバーの 📁 ファイルアイコン をクリック → 「**アップロード**」ボタン

| ファイル名 | 内容 | 入手先 |
|---|---|---|
| `R_libs.zip` | R パッケージ群 | 先生から配布 |
| `JIAsyno_CITEseq_T_share.rds` | 解析データ | 先生から配布 |

> ⚠️ アップロードには数分かかります。両方のファイルが `/content/` に表示されるまで待ってください。

### ステップ③：セル1 → セル2 の順に実行
アップロード完了後、上から順にセルを実行してください。


---
## 📦 セル1：環境セットアップ（毎回実行 — 約2〜3分）

アップロードした `R_libs.zip` を展開して R パッケージを読み込みます。

> ✅ アップロード済みの `R_libs.zip` が左のファイル一覧に表示されていることを確認してから実行してください。


In [ ]:
# ============================================================
# 環境セットアップ：R_libs.zip を展開してパッケージを読み込む
# ============================================================

R_LIBS_LOCAL <- "/content/R_libs"

# --- Step 1: R_libs.zip を展開（未展開の場合のみ）---
if (!dir.exists(R_LIBS_LOCAL)) {
  zip_path <- "/content/R_libs.zip"
  if (!file.exists(zip_path)) {
    stop("❌ R_libs.zip が見つかりません。\n",
         "   左サイドバーの「アップロード」ボタンから R_libs.zip をアップロードしてください。")
  }
  cat("📦 R_libs.zip を展開中...（数分かかります）\n")
  system(paste("unzip -q", zip_path, "-d /content/"))
  cat("✅ 展開完了！\n")
} else {
  cat("✅ R_libs はすでに展開済みです。\n")
}

# --- Step 2: ライブラリパスを設定 ---
.libPaths(c(R_LIBS_LOCAL, .libPaths()))
cat("📁 ライブラリパス:", .libPaths()[1], "\n")

# --- Step 3: ライブラリ読み込み ---
cat("\n📚 ライブラリを読み込み中...\n")
suppressMessages({
  library(Seurat)
  library(magrittr)
  library(patchwork)
  library(dplyr)
  library(ggplot2)
})

set.seed(123)
cat("\n✅ セットアップ完了！ Seurat バージョン:", as.character(packageVersion("Seurat")), "\n")


---
## 📁 セル2：データの読み込み

アップロードした `JIAsyno_CITEseq_T_share.rds` を読み込みます。

> ✅ 左のファイル一覧に `JIAsyno_CITEseq_T_share.rds` が表示されていることを確認してから実行してください。


In [ ]:
# ============================================================
# T細胞データの読み込み
# ============================================================
DATA_FILE <- "/content/JIAsyno_CITEseq_T_share.rds"

if (!file.exists(DATA_FILE)) {
  stop("❌ JIAsyno_CITEseq_T_share.rds が見つかりません。\n",
       "   左サイドバーの「アップロード」ボタンからファイルをアップロードしてください。")
}

cat("🔄 T細胞データを読み込み中...\n")
cat("   ファイルサイズ:", round(file.size(DATA_FILE) / 1e6, 1), "MB\n")
t_cells <- readRDS(DATA_FILE)

cat("✅ データ読み込み完了！\n")
t_cells


---
# イントロダクション：サブクラス解析の重要性

Part 1では、データセット全体から「T細胞」という大きなグループを特定し、抽出しました。
しかし、一口にT細胞と言っても、その中には免疫を司令する**ヘルパーT細胞(CD4+)**、攻撃を担う**キラーT細胞(CD8+)**、過剰な免疫を抑える**制御性T細胞(Treg)**など、多様な役割を持つ細胞が混ざっています。

### なぜ再計算が必要なのか？

全体解析で見つかった「高変動遺伝子（Variable Features）」は、T細胞とB細胞やマクロファージを分けるための遺伝子が中心です。
T細胞の内部にあるわずかな違いを浮き彫りにするためには、**T細胞の中だけで遺伝子のばらつきを計算し直す**必要があります。

---
# サブセットに対する再解析パイプライン

T細胞集団に対して、改めて「特徴探し」から「分類」までを行います。

## ステップ1：高変動遺伝子の再選出

T細胞のサブタイプ（CD4, CD8, Tregなど）を区別するために重要な遺伝子を新たに選びます。

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4)

# ターゲットをRNAに設定
DefaultAssay(t_cells) <- "RNA"

# T細胞の中での高変動遺伝子を特定
t_cells <- FindVariableFeatures(t_cells, selection.method = "vst", nfeatures = 2000)

# 上位の変動遺伝子をプロット
top10_t <- head(VariableFeatures(t_cells), 10)
LabelPoints(plot = VariableFeaturePlot(t_cells), points = top10_t, repel = TRUE)

## ステップ2：スケーリングとPCA（主成分分析）

T細胞内の差異に基づいた新しい「軸（主成分）」を作成します。

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 5)

t_cells <- ScaleData(t_cells)
t_cells <- RunPCA(t_cells, features = VariableFeatures(object = t_cells))

# 実際の主成分をプロット
## ここでの色はT細胞以外の細胞を含めてクラスタリングしたときの情報です
DimPlot(t_cells, reduction = "pca") + NoLegend()

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 5)

# どの主成分までが重要か、エルボープロットで確認
ElbowPlot(t_cells)

## ステップ3：クラスタリングとUMAPによる可視化

T細胞の中での「似た者同士」をグループ化します。

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)

# PC1から10までを使用して計算
t_cells <- FindNeighbors(t_cells, dims = 1:10)

# resolution（解像度）を調整することで、クラスターの細かさを変えられます
# 今回は少し細かめに 0.8 に設定してみます
t_cells <- FindClusters(t_cells, resolution = 0.8)

# UMAPの計算
t_cells <- RunUMAP(t_cells, dims = 1:10)

# 結果の描画
DimPlot(t_cells, reduction = "umap", label = TRUE)

---
# 生物学的な解釈：このT細胞は何者か？

各クラスターがどのようなT細胞なのか、代表的なマーカー遺伝子の発現を見てみましょう。

| マーカー | 細胞種 |
|---|---|
| **CD4** | ヘルパーT細胞 |
| **CD8A** | キラーT細胞（細胞傷害性T細胞） |
| **CCR7** | ナイーブT細胞 |
| **FOXP3** | 制御性T細胞 (Treg) |
| **KLRB1** | メモリーT細胞マーカー |
| **TRAV1-2 / SLC4A10** | MAIT細胞 |
| **CXCL13** | Follicular helper T細胞 (Tfh) / Peripheral helper T細胞 (Tph) |
| **NCAM1 / FCGR3A** | NK細胞 |
| **GZMB / PRF1** | 細胞傷害活性を持つ細胞 |
| **GZMK** | 細胞傷害性T細胞-2 |
| **TRDV2 / TRGV9** | γδ T細胞 |
| **ZNF683 / KLRC2** | 組織常在性メモリーT細胞 (TRM) |

> 💡 **T細胞アノテーションのコツ**
> まず「CD4陽性」と「CD8陽性」に大別し、その後TregやMAIT、NKTなどの特殊サブタイプを探すと整理しやすいです。

> 💡 **なぜNK細胞のマーカーを見るのか？**
> NK細胞はT細胞と共通の前駆細胞から分化し、遺伝子発現が連続的に繋がっているため、T細胞集団に混入していることがあります。

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 28)

# マーカー遺伝子の発現をUMAP上にプロット
features_to_plot <- c("CD3E", "CD4", "CD8A", "CCR7", "FOXP3", "KLRB1", 
                      "TRAV1-2", "SLC4A10", "CXCL13", 
                      "NCAM1", "FCGR3A", 
                      "GZMB", "PRF1", "GZMK", 
                      "TRDV2", "TRGV9", 
                      "ZNF683", "KLRC2")
FeaturePlot(t_cells, features = features_to_plot, ncol = 4)

### クラスターごとの発現比較（バイオリンプロット）

数値的な違いをより明確に確認します。

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 36)
VlnPlot(t_cells, features = features_to_plot, pt.size = 0, ncol = 2)

### データドリブンなマーカー遺伝子の探索

既存のマーカー遺伝子発現以外に、各クラスターに特徴的な遺伝子を統計的に探します。

In [ ]:
# 各クラスターの特徴的なマーカーを検出（少し時間がかかります）
t_markers <- FindAllMarkers(t_cells, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

# 各クラスターのTop8遺伝子を表示
t_markers %>%
  dplyr::group_by(cluster) %>%
  dplyr::top_n(n = 8, wt = avg_log2FC) %>%
  as.data.frame()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 15)

DoHeatmap(t_cells, features = t_markers %>% 
            dplyr::group_by(cluster) %>% 
            dplyr::top_n(n = 8, wt = avg_log2FC) %>%
            dplyr::pull(gene) %>%
            unique())

---
# 🎓 実習パート：クラスターの同定とラベル貼り

解析の結果をもとに、クラスター番号を生物学的な名前に書き換えます。

**手順：**
1. 上の FeaturePlot と VlnPlot を参考に、各クラスターのT細胞種を考えてください
2. 下の `new_cluster_names` の `?????` を自分の考えに基づいて書き換えてください
3. セルを実行して結果を確認してください

| クラスター番号 | 主なマーカー | 想定されるT細胞種 |
|---|---|---|
| 0 | ? | ? |
| 1 | ? | ? |
| 2 | ? | ? |
| ... | ... | ... |

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)

# 現在のクラスター数を確認
Idents(t_cells) <- t_cells$seurat_clusters
cat("現在のクラスター:", levels(t_cells), "\n")

# ============================================================
# ↓↓↓ ここを自分で考えて書き換えよう！↓↓↓
# ============================================================
new_cluster_names <- c(
  "0"  = "?????",   # ヒント: どのマーカーが高い？
  "1"  = "?????",
  "2"  = "?????",
  "3"  = "?????",
  "4"  = "?????",
  "5"  = "?????",
  "6"  = "?????",
  "7"  = "?????",
  "8"  = "?????",
  "9"  = "?????",
  "10" = "?????",
  "11" = "?????",
  "12" = "?????",
  "13" = "?????",
  "14" = "?????",
  "15" = "?????",
  "16" = "?????",
  "17" = "?????"
)
# ============================================================

# SeuratオブジェクトのIdents（クラスター名）を書き換え
t_cells <- RenameIdents(t_cells, new_cluster_names)

# 書き換えた名前でUMAPを表示
DimPlot(t_cells, reduction = "umap", label = TRUE, pt.size = 0.5, repel = TRUE) + 
  NoLegend() +
  ggtitle("T細胞サブクラスター（自分のアノテーション）")

---
# 💾 結果の保存

解析結果をGoogle Driveに保存します。

In [ ]:
# ============================================================
# 結果をGoogle Driveに保存する（任意）
# ============================================================
# まずDriveをマウント
system("python3 -c 'from google.colab import drive; drive.mount(chr(34) + chr(47) + chr(99) + chr(111) + chr(110) + chr(116) + chr(101) + chr(110) + chr(116) + chr(47) + chr(100) + chr(114) + chr(105) + chr(118) + chr(101) + chr(34))' ")

output_dir <- "/content/drive/MyDrive/scRNA実習_結果"
dir.create(output_dir, showWarnings = FALSE)

save_path <- file.path(output_dir, "JIAsyno_CITEseq_T_annotated.rds")
saveRDS(t_cells, save_path)
cat("✅ 保存完了:", save_path, "\n")

---
# まとめと考察

サブクラス解析を行うことで、全体の解析では一つの「T細胞」という塊だったものが、実は多様な集団の集まりであることが分かりました。
自己免疫疾患（今回はJIA）の炎症部位において、**「どのT細胞サブセットが特に活性化しているのか」**、あるいは**「治療によってどの集団が減るのか」**を詳細に追跡することが、病態理解の鍵となります。

---

## 📝 学生への課題

1. `resolutionを` 0.2 や 1.5 に変えると、クラスターの数はどう変化しますか？
2. あなたがアノテーションしたクラスターの中で、最も確信が持てなかったのはどのクラスターですか？その理由を書いてください。
3. JIA（若年性特発性関節炎）の病態において、炎症滑膜で増加していると予想するT細胞サブセットはどれですか？その理由を文献を参考に考察してください。

---
# 🔍 参考：resolution を変えて試してみよう

クラスター解像度を変えると結果がどう変わるか実験してみましょう。

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)

# ============================================================
# ↓ resolution の値を変えて試してみよう！
# ============================================================
res_to_compare <- c(0.2, 0.8, 1.5)
# ============================================================

plot_list <- list()
for (res in res_to_compare) {
  t_temp <- FindClusters(t_cells, resolution = res)
  Idents(t_temp) <- t_temp$seurat_clusters
  p <- DimPlot(t_temp, reduction = "umap", label = TRUE) + 
    NoLegend() + 
    ggtitle(paste0("resolution = ", res, "\n(クラスター数: ", length(levels(t_temp)), ")"))
  plot_list[[as.character(res)]] <- p
}

wrap_plots(plot_list, ncol = 3)

---
# セッション情報（環境情報の記録）

In [ ]:
sessionInfo()

In [ ]:
# データセットのサイズ確認 (MB)
cat("t_cells のサイズ:", round(object.size(t_cells) / 1e6, 1), "MB\n")